### 워크플로우 이니시

In [1]:
# 특정 워크플로우의 초기는 다음과 같이 생긴다

from entity.process import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole, Process
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs
import os
import logging
from core import logging_utils, utils

log_levels = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR,
    "CRITICAL": logging.CRITICAL
}

log_level = log_levels.get(os.environ.get("LOG_LEVEL", "INFO").upper(), logging.INFO)

logging_utils.setup_logging(
    log_dir="logs",
    log_level=log_level,
    max_bytes=10 * 1024 * 1024, 
    backup_count=5, 
    console_output=True 
)

logging_utils.cleanup_old_logs(log_dir="logs", days_to_keep=7)


12:47:17 - core.logging_utils - INFO - Logging initialized - logs directory: c:\Users\kakao\Desktop\AI_Agent\Semantic Layer\logs
12:47:17 - core.logging_utils - INFO - Log rotation configured: max 10.0MB per file, keeping 5 backups


In [2]:
resource_db = load_resource_specs("./ResourceSpec/TokenSpec.yaml")
validator = TokenValidator(resource_db)

# Chain Validator 인스턴스 생성
chain_check = SpecChainValidator(validator)

FileNotFoundError: [Errno 2] No such file or directory: './ResourceSpec/TokenSpec.yaml'

In [4]:
### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_A = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_001",
    description="금융 뉴스 감성 분석",
    type=TaskType.PYTHON_FUNC, 
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V1",
    output_spec_id="RS_RISK_TOKEN_V2"
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_B = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_002",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2",

)

task_C = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_003",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2"

)

task_D = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_004",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2"
)


### 프로세스 테스트

In [17]:
# 최초 선언시 아이디 선언
process_test = Process("ojs_test_process")

# (ORANGE) 나중에 Process 선언문 읽고 아래 action이 자동 수행되는 로직이 필요
process_test.tasks
process_test.add_task(task_A)
process_test.add_task(task_B)
process_test.add_link(task_A, task_B)

# 그래프 구조 완성된거 확인 가능
process_test.graph
process_test.compile(chain_validator=chain_check)

proc = Process("RISK_MODEL_FLOW")

# 2. 태스크 등록
proc.add_task(task_A) # 재무
proc.add_task(task_B) # 거시
proc.add_task(task_C) # 통합 모델
proc.add_task(task_D) # 발표 모델
proc.add_link(task_A, task_C) # 재무 -> 통합
proc.add_link(task_B, task_C) # 거시 -> 통합
proc.add_link(task_C, task_D)
proc.compile(chain_check)

09:34:29 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_001 -> TASK_TEST_002
09:34:29 - Proc_ojs_test_process - INFO - [COMPILE SUCCESS] Spec 일치 & Process Topology 정상.
09:34:29 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_001 -> TASK_TEST_003
09:34:29 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_002 -> TASK_TEST_003
09:34:29 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_003 -> TASK_TEST_004
09:34:29 - Proc_RISK_MODEL_FLOW - INFO - [COMPILE SUCCESS] Spec 일치 & Process Topology 정상.
09:34:29 - Proc_RISK_MODEL_FLOW - INFO - 토큰 주입 = TASK_TEST_001


### /engine/engine.py

In [ ]:
import importlib
import logging
from dataclasses import dataclass, field
from typing import Any, Optional, Dict, List
from datetime import datetime

# [Dependency] FiringResult
@dataclass
class FiringResult:
    task_id: str
    success: bool
    message: str
    new_token: Optional[Any] = None
    elapsed_ms: float = 0.0
    routes_triggered: int = 0

# 사용자 정의 에러: 엔진 차원에서 토큰 자체를 거부할 때 사용
class TokenIntegrityError(Exception):
    pass

class ExecEngine:
    """
    핵심 프로세스 실행 엔진
    CSPN의 Transition(Task)을 실행하고, 토큰을 변환/생성하는 역할 수행
    
    내부 절차 : 가드 체크 -> 입력 확인 -> 함수 실행 -> 아웃풋 확인 -> 토큰 갱신 -> 라우팅
    """
    def __init__(self, token_validator, ttl_seconds: int = 3600):
        # Init 시점에 Token Content Validator 주입
        self.tv = token_validator
        self.logger = logging_utils.get_logger(f"Engine")
        self.ttl_seconds = ttl_seconds

    def run_step(self, process: Process) -> Optional[FiringResult]:
        """
        한 스텝의 태스크를 꺼내 처리함
        """
        # 1. 프로세스 큐에서 Job를 추출
        if not process.token_queue:
            self.logger.warning(f"[RUN] 처리할 토큰이 큐에 없습니다.")
            return None
        
        # 2. 큐에서 토큰과 태스크 아이디 추출
        target_task_id, token = process.token_queue.popleft()
        task = process.tasks[target_task_id]

        start_time = datetime.now()

        # 3. 토큰 엔벨로프 Validation (콘텐츠 외부 무결성)
        try:
            self._validate_envelope(token)
        except TokenIntegrityError as e:
            self.logger.error(f"Token Integrity Check Failed: {e}")
            return FiringResult(task.task_id, False, f"Token Integrity Fail: {str(e)}")

        # 4. 태스크 실행 전 가드 체크
        if not self._check_guards(task, token):
            return FiringResult(task.task_id, False, message="Guard Condition Failed")
        
        # 5. 토큰 콘텐츠 Validation (콘텐츠 내부 무결성)
        try:
            self.tv.validate(token.content, task.input_spec_id)
        except Exception as e:
            # SemanticError, ValueError(Spec 없음) 등 모든 검증 에러 포착   
            self.logger.warning(f"Task {task.task_id} Input Validation Failed: {e}")
            return FiringResult(task.task_id, False, f"Input Spec Fail: {str(e)}")
        
        # 6. 동적 실행
        # task.target은 해당 태스크에 할당된 function 혹은 API 실행을 의미
        try:
            func = self._resolve_function(task.target)
            
            # 실행 : Token Content + task config 주입
            output_content = func(token.content, **task.config)

        except Exception as e:
            self.logger.error(f"Task {task.task_id} Execution Logic Failed: {e}", exc_info=True)
            return FiringResult(task.task_id, False, f"Runtime Execution Error: {str(e)}")

        # 7. [Validator] Output Spec Validation
        try:
            self.tv.validate(output_content, task.output_spec_id)
        except Exception as e:
            self.logger.error(f"Task {task.task_id} Output Validation Failed: {e}", exc_info=True)
            return FiringResult(task.task_id, False, f"Output Spec Fail: {str(e)}")

        # 8. Token Evolution (State Update)
        new_token = self._evolve_token(token, output_content, task)

        # 9. Routing (PetriNet Propagation)
        # Process에게 토큰 도착 알림 (Process 내부 로직으로 병합/대기 수행)
        routes_count = self._propagate_token(process, task, new_token)

        elapsed = (datetime.now()-  start_time).total_seconds() * 1000
        return FiringResult(task.task_id, True, "Success", new_token, elapsed, routes_count)

    ### ===== 내부 로직 ===== ###

    def _check_guards(self, task: Any, token: Any) -> bool:
        """Topic 가중치 기반 실행 조건 평가"""
        if not task.guards:
            return True
        
        token_topics = getattr(token, 'topics', {})
        for guard in task.guards:
            score = token_topics.get(guard.target_topic_id, 0.0)
            if score < guard.min_relevance:
                self.logger.debug(f"Guard Fail: {guard.target_topic_id} ({score} < {guard.min_relevance})")
                return False
        return True

    def _resolve_function(self, target_path: str):
        """'module.path:func_name' 문자열을 실제 함수 객체로 변환"""
        try:
            module_path, func_name = target_path.split(":")
            module = importlib.import_module(module_path)
            return getattr(module, func_name)
        except (ValueError, ImportError, AttributeError) as e:
            raise ImportError(f"Function resolution failed for '{target_path}': {e}")

    def _evolve_token(self, old_token: Any, new_content: Dict, task: Any) -> Any:
        """이전 토큰 승계 및 이력 업데이트"""
        # Token 클래스 정의에 따라 model_copy 또는 생성자 사용
        # 예시: Pydantic model_copy
        new_history = getattr(old_token, 'history', []) + [task.task_id]
        
        return old_token.model_copy(update={
            "content": new_content,
            "history": new_history
        })

    def _propagate_token(self, process: Any, current_task: Any, token: Any) -> int:
        """
        [Routing Logic]
        다음 Task들을 찾아 'Process.arrive_token'을 호출함.
        """
        next_tasks = process.get_next_nodes(current_task.task_id)
        
        if not next_tasks:
            process.completed_tokens.append(token) # End of Chain
            return 0

        triggered = 0
        for next_task in next_tasks:
            # Look-ahead Guard Check: 갈 자격이 있는 경로인가?
            if self._check_guards(next_task, token):
                # [핵심] Process의 Place 로직 호출 (Petri Net 동기화 위임)
                process.arrive_token(
                    from_task_id=current_task.task_id,
                    to_task_id=next_task.task_id,
                    token=token
                )
                triggered += 1
            else:
                self.logger.info(f"Route Ignored: {current_task.task_id} -> {next_task.task_id}")
        
        return triggered

    def _validate_envelope(self, token) -> None:
            """
            [Layer 1 Validation]
            토큰의 내용물(Content)을 보기 전, 토큰 자체의 무결성(Envelope)을 검증.
            실패 시 즉시 TokenIntegrityError 발생시키고 프로세스 중단.
            """
            # 1. 식별자 검증 (Traceability Check)
            if not token.trace_id or not isinstance(token.trace_id, str):
                raise TokenIntegrityError(f"Invalid Trace ID: {token.trace_id}")

            # 2. 메타데이터 무결성 검증 (Metadata Sanity Check)
            if token.topics:
                for topic, score in token.topics.items():
                    if not (0.0 <= score <= 1.0):
                        raise TokenIntegrityError(f"Topic score out of range [0,1]: {topic}={score}")

            # 3. 생존 시간 검증 (Time-to-Live Check)
            elapsed = (datetime.now() - token.created_at).total_seconds()
            if elapsed > self.ttl_seconds:
                raise TokenIntegrityError(f"Token Expired (Zombie Token). Elapsed: {elapsed:.2f}s > Limit: {self.ttl_seconds}s")
            
            self.logger.info(f"[INFO] Token {token.trace_id} envelope is intact.")

### Engine Run Example

#### run_process 정의

In [ ]:
def run_process(self, process: Process, max_steps: int = 20) -> str:
        """
        [Auto-Pilot] 프로세스가 종료되거나 교착상태(Deadlock)에 빠질 때까지 연속 실행
        """
        steps = 0
        while steps < max_steps:
            # 1. 한 스텝 실행
            result = self.run_step(process)
            
            # 2. 종료 조건 체크
            if result is None:
                self.logger.info("No more enabled transitions. (Finished or Deadlock)")
                return "STOPPED"
            
            steps += 1
            
        self.logger.warning("Max steps reached. Possible infinite loop.")
        return "MAX_STEPS_REACHED"

#### 실행 예시

In [25]:
Exec_Engine = ExecEngine(validator)

In [27]:
# 최초 토큰 생성
valid_token = Token(
    trace_id = "dfhou3898dfalss28fhs", # 향후 랜덤 생성기 필요
    content = {
        "text" : "Loan Holder A Risk Analysis",
        "risk_score" : 0.96,
        "recent_risk" : "최근 82일간 대출 연체 기록이 있습니다."
    },
    topics = {
        "TOPIC_FINANCE" : 0.8
    }
)

# 주입
proc.inject_token("TASK_TEST_001", valid_token)

Exec_Engine.run_step(proc)
# FiringResult(task_id='TASK_TEST_001', success=False, 
# message='Guard Condition Failed', new_token=None, elapsed_ms=0.0, routes_triggered=0)

09:38:49 - Proc_RISK_MODEL_FLOW - INFO - 토큰 주입 = TASK_TEST_001
09:38:49 - Engine - INFO - [INFO] Token dfhou3898dfalss28fhs envelope is intact.
09:38:49 - Validator - INFO - 토큰 검증 시작 (Spec: RS_RISK_TOKEN_V1)
09:38:49 - Engine - ERROR - Task TASK_TEST_001 Execution Logic Failed: Function resolution failed for 'utils.analysis:calculate_sentiment': No module named 'utils'
Traceback (most recent call last):
  File "C:\Users\kakao\AppData\Local\Temp\ipykernel_22312\1597196177.py", line 115, in _resolve_function
    module = importlib.import_module(module_path)
  File "c:\Users\kakao\miniforge3\envs\agent_env\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call

FiringResult(task_id='TASK_TEST_001', success=False, message="Runtime Execution Error: Function resolution failed for 'utils.analysis:calculate_sentiment': No module named 'utils'", new_token=None, elapsed_ms=0.0, routes_triggered=0)